In [1]:
import shutil
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
!pip install ipygany

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.0 MB/s eta 0:00:00


In [2]:
# Install necessary libraries
!pip install torch pandas scikit-learn
!apt-get install -y wget unzip

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
# Clone the PointConv repository
!git clone https://github.com/DylanWusee/pointconv_pytorch.git

# Change directory to the project folder
%cd pointconv_pytorch

Cloning into 'pointconv_pytorch'...
remote: Enumerating objects: 63, done.
remote: Total 63 (delta 0), reused 0 (delta 0), pack-reused 63
Receiving objects: 100% (63/63), 22.56 KiB | 11.28 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Error downloading object: checkpoints/checkpoint.pth (4a526dd): Smudge error: Error downloading checkpoints/checkpoint.pth (4a526dd7dae44ebd6b9c2b6a9a7eac6bb6608b88d04c74ed11b93adb909e9bb4): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/pointconv_pytorch/.git/lfs/logs/20240702T001930.211533129.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: checkpoints/checkpoint.pth: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

/content/pointconv_pytorch


In [4]:
# Download ModelNet40 dataset with --no-check-certificate option
!wget --no-check-certificate https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip

--2024-07-02 00:19:30--  https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 1705117335 (1.6G) [application/zip]
Saving to: ‘modelnet40_normal_resampled.zip’

modelnet40_normal_r 100%[===================>]   1.59G  18.4MB/s    in 97s     

2024-07-02 00:21:07 (16.8 MB/s) - ‘modelnet40_normal_resampled.zip’ saved [1705117335/1705117335]



In [5]:
# Unzip the dataset and place it in the correct directory
!unzip modelnet40_normal_resampled.zip -d data/

Streaming output truncated to the last 5000 lines.
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0412.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0473.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0258.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0663.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0384.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0362.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0048.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0344.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0027.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0288.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0682.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0432.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0702.txt  
  inflating: data/modelnet40_normal_resampled/sofa/sofa_0083.txt  
  inflating

In [6]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU is enabled.")
else:
    print("CUDA is not available. Please enable GPU in your runtime settings.")

CUDA is available. GPU is enabled.


In [8]:
!python train_cls_conv.py --model pointconv_modelnet40 --normal --epoch 10

The size of train data is 9843
The size of test data is 2468
No existing model, starting training from scratch...
Epoch 1 (1/10):
100% 308/308 [03:38<00:00,  1.41it/s]
Train Accuracy: 0.187714
 Loss: 2.844346
 Test Accuracy: 0.235413   ***  Best Accuracy: 0.000000
Epoch 2 (2/10):
100% 308/308 [03:34<00:00,  1.43it/s]
Train Accuracy: 0.324553
 Loss: 2.953638
 Test Accuracy: 0.365478   ***  Best Accuracy: 0.000000
Epoch 3 (3/10):
100% 308/308 [03:40<00:00,  1.40it/s]
Train Accuracy: 0.445932
 Loss: 2.432372
 Test Accuracy: 0.544571   ***  Best Accuracy: 0.000000
Epoch 4 (4/10):
100% 308/308 [03:34<00:00,  1.44it/s]
Train Accuracy: 0.545940
 Loss: 2.101863
 Test Accuracy: 0.664506   ***  Best Accuracy: 0.000000
Epoch 5 (5/10):
100% 308/308 [03:36<00:00,  1.43it/s]
Train Accuracy: 0.602225
 Loss: 2.261882
 Test Accuracy: 0.731767   ***  Best Accuracy: 0.000000
Epoch 6 (6/10):
100% 308/308 [03:38<00:00,  1.41it/s]
Train Accuracy: 0.633881
 Loss: 1.158255
 Test Accuracy: 0.771880   ***  Best

In [9]:
!python eval_cls_conv.py --checkpoint /content/pointconv_pytorch/experiment/pointconv_modelnet40_ModelNet40-2024-07-02_00-29/checkpoints/pointconv_modelnet40-0.857780-0010.pth --normal

The size of test data is 2468
Load CheckPoint...
Start evaluating...
100% 78/78 [01:05<00:00,  1.18it/s]
Total Accuracy: 0.850962
